<a href="https://colab.research.google.com/github/Alice-zou/DaisyAI-Developers/blob/main/Disaster_chatbot_Zephyr_7B_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain langchain-community sentence-transformers gradio pypdf PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import os
import gradio as gr
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
model_name = "anakin87/zephyr-7b-alpha-sharded"

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [ ]:
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1
    return tokenizer

In [ ]:
model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

stop_token_ids = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/NeurIPS Outreach/Dataset/Dominick Pelaia - Wildfires'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests

# Outputs a dictionary of recent natural disasters.
def all_hazards():
	disasters = []
	main_api = "https://gdacs.org/xml/gdacs_archive.geojson"
	json_data = requests.get(main_api).json()
	for item in json_data['features']:
		disaster = {}
		description = item['properties']['htmldescription']
		details = {
		'eventname':item['properties']['name'],
		'type':item['properties']['eventtype'],
		'coordinates':[round(i,4) for i in item['geometry']['coordinates']][::-1],
		'countries':item['properties']['country'],
		'level':item['properties']['alertlevel'],
		'active':item['properties']['iscurrent'],
		'geometry':item['properties']['url']['geometry'],
		'casualties':item['properties']['severitydata']['severity'],
		'glide': item['properties']['glide'],
		'details': item['properties']['url']['details'],
		'scores': item['properties']['alertscore'],
		'from_date': item['properties']['fromdate'][0:10],
		'to_date': item['properties']['todate'][0:10],
		'severity': item['properties']['severitydata']['severity'],
		'severity_unit': item['properties']['severitydata']['severityunit'],
		}

		disaster[description] = details
		disasters.append(disaster)
	return disasters

In [ ]:
# Load PDF files from a folder. These files will be referenced when chatbot generate answers
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()
print(documents)

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

# Embedding text chunks
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

# Create a vector database
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

# Create a retriever so that query the database above and retrieve relevant text chunks based on a user query.
retriever = vectordb.as_retriever()

[Document(page_content='6/21/24, 1:57\u202fPMMaui wildfires of 2023 -- Britannica Online Encyclopedia\nPage 1 of 4https://www.britannica.com/print/article/2229349\nMaui wildﬁres of 2023Lahaina, Maui, Hawaii, after a wildﬁrereduced the historic resort town to ashand ruins in August 2023.Maui wildﬁres of 2023Maui wildﬁres of 2023, a series of wildﬁres that burned parts ofthe island of Maui in the U.S. state of Hawaii in August 2023.The ﬁres, which began on August 8, struck hardest the historicresort town of Lahaina, on Maui’s western peninsula, reducingmost of the town to ash and ruins. 98 people were killed inLahaina by the smoke and ﬂames or by drowning, making thewildﬁre one of the world’s deadliest on record. Almost 3,000structures were reported to have been either damaged ordestroyed by the ﬁre. In addition to the ﬁres on Maui, a series ofless devastating wildﬁres burned parts of the island of Hawaiistarting on August 9.CausesWhile local and state government ofﬁcials have hesitated 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# initialize a text generation pipeline using a specified model and tokenizer.
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# Wraps the text generation pipeline in a Hugging Face pipeline object
llm = HuggingFacePipeline(pipeline=pipeline)

# Defines a function that takes a query string and a chat history list as input.
# Returns a tuple containing the updated chat history.
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        # Initializes a conversational retrieval chain using the provided language model
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )
        # Executes the QA chain with the provided query and chat history. The result contains the generated response.
        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history

    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

ValueError: The following `model_kwargs` are not used by the model: ['model', 'device_map'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
# use the Gradio library to create a web-based chatbot interface
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your data (Zephyr 7B Alpha)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

# In colab environment, need to set 'share=True'. So that the generated public URL can be accessed.
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://db4690b2ddebb434f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
